In [110]:
# import the package
import numpy as np
import pandas as pd
#from geopy.geocoders import Nominatim

###  Goal: find relationship between the different dataset and prepare to import into database


In [132]:
# load the extract data source
# dataset1
usairbnb2020 = pd.read_csv("./Extract_Data/usairbnb2020.csv")
usairbnb2020 = usairbnb2020.loc[:, ~usairbnb2020.columns.str.contains('^Unnamed')]
# dataset2
weather = pd.read_csv("./Extract_Data/weather.csv")
weather = weather.loc[:, ~weather.columns.str.contains('^Unnamed')]
# dataset3 
state = pd.read_csv("./Extract_Data/state.csv")
state = state.loc[:, ~state.columns.str.contains('^Unnamed')]
# dataset4
usairbnb2022 = pd.read_csv("./Extract_Data/usairbnb2022.csv")
usairbnb2022 = usairbnb2022.loc[:, ~usairbnb2022.columns.str.contains('^Unnamed')]
# dataset5
crime = pd.read_csv("./Extract_Data/crime.csv")
crime = crime.loc[:, ~crime.columns.str.contains('^Unnamed')]
# dataset6
OSMR = pd.read_csv("./Extract_Data/OSMR_df.csv")
OSMR = OSMR.loc[:, ~OSMR.columns.str.contains('^Unnamed')]

# dataset7
# minwage2020 = pd.read_csv("./Extract_Data/minwage_2020.csv")
# minwage2020 = minwage2020.loc[:, ~minwage2020.columns.str.contains('^Unnamed')]

##  Let's clean and link the datasets


In [4]:
usairbnb2020.head()

,id,name,host_id,host_name,latitude,longitude,city,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,35.65146,-82.62792,Asheville,Private room,60,1,138,16/02/20,1.14,1,0
1,80905,French Chic Loft,427027,Celeste,35.59779,-82.55540,Asheville,Entire home/apt,470,1,114,07/09/20,1.03,11,288
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,35.60670,-82.55563,Asheville,Entire home/apt,75,30,89,30/11/19,0.81,2,298
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul,35.57864,-82.59578,Asheville,Entire home/apt,90,1,267,22/09/20,2.39,5,0
4,160594,Historic Grove Park,769252,Elizabeth,35.61442,-82.54127,Asheville,Private room,125,30,58,19/10/15,0.52,1,0


# Dataset 1

In [82]:
# data1: id and host id data 
ID_df = usairbnb2020[["id","name","host_id","host_name"]]
ID_df = ID_df.rename(columns = {"name":"id_name"})
ID_df.head()
#ID_df.to_csv("ID.csv", index = True, header = True)

,id,id_name,host_id,host_name
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne
1,80905,French Chic Loft,427027,Celeste
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul
4,160594,Historic Grove Park,769252,Elizabeth


In [67]:
# check the unique
sum(ID_df.id.value_counts()!= 1)

0

In [70]:
ID_df.host_id.value_counts()

48005494     653
8534462      291
229095817    272
30787515     206
2154262      199
            ... 
301725621      1
81955386       1
213366195      1
26698095       1
367917574      1
Name: host_id, Length: 108039, dtype: int64

# Dataset 2

In [84]:
# data2: room info
room_df = usairbnb2020[["id","room_type","price","minimum_nights","availability_365","number_of_reviews","reviews_per_month"]]
room_df.shape # (177394, 7)
room_df.head()
#room_df.to_csv("room.csv", index = True, header = True)

# Dataset 3 

In [55]:
# datasets3: add the year, zip code and state 
year = [2020] * 177394
usairbnb2020["year"] = year
connect_df = pd.DataFrame({
    "state":state["State"],
    "city" : weather["City"],
    "zipcode":state["Zip Code"]
})
# merge this data to the main dataset
usairbnb2020_new = pd.merge(usairbnb2020,connect_df, on = "city")
usairbnb2020_new = usairbnb2020_new[["year","id","host_id","city","state","latitude","longitude","room_type","price","minimum_nights","number_of_reviews","last_review","reviews_per_month","calculated_host_listings_count","availability_365"]]
usairbnb2020_new.head()
#usairbnb2020_new.to_csv("usairbnb2020.csv", index = True, header = True)

,year,id,host_id,city,state,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2020,38585,165529,Asheville,North Carolina,35.65146,-82.62792,Private room,60,1,138,16/02/20,1.14,1,0
1,2020,80905,427027,Asheville,North Carolina,35.59779,-82.55540,Entire home/apt,470,1,114,07/09/20,1.03,11,288
2,2020,108061,320564,Asheville,North Carolina,35.60670,-82.55563,Entire home/apt,75,30,89,30/11/19,0.81,2,298
3,2020,155305,746673,Asheville,North Carolina,35.57864,-82.59578,Entire home/apt,90,1,267,22/09/20,2.39,5,0
4,2020,160594,769252,Asheville,North Carolina,35.61442,-82.54127,Private room,125,30,58,19/10/15,0.52,1,0


# Dataset 4

In [95]:

weather = weather.rename(columns = {"City":"city",
                                   "Lat":"lat",
                                   "Lng":"lng",
                                   "Temp":"temputature",
                                   "Humidity":"humidity",
                                   "Cloudiness":"cloudiness",
                                   "Wind Speed":"wind_speed"})
weather.head(30)

#weather.to_csv("weather.csv", index = True, header = True)

# Dataset 5

In [94]:
state = state.rename(columns = {"State":"state",
                               "Country":"country",
                               "Zip Code":"zip_code"})
state["id"] = state.index + 1
state.set_index("id")

state.head()
# state.to_csv("state.csv", index = True, header = True)

,state,country,zip_code,id
0,North Carolina,us,28801,1
1,Texas,us,78703,2
2,Massachusetts,us,2118,3
3,Florida,us,33315,4
4,Massachusetts,us,2139,5


# Dataset 6

In [33]:
# datasets 6
# we may add this in to mongodb
# add the year column 
#usairbnb2022.shape #dim(3120, 5)
year = [2022] * 3120
usairbnb2022["year"] = year
usairbnb2022_new = usairbnb2022[["year","City","Name","Price","Bedtype","Review"]]
usairbnb2022_new.head()
#usairbnb2022_new.to_csv("usairbnb2022.csv", index = True, header = True)

,year,City,Name,Price,Bedtype,Review
0,2022,Asheville,"Cabin in 1, South Marshall",170,1 queen bed,4.99 (252)
1,2022,Asheville,Cabin in Hendersonville,249,1 king bed,5.0 (144)
2,2022,Asheville,Chalet in Clyde,176,3 beds,5.0 (8)
3,2022,Asheville,Treehouse in Fletcher,132,1 queen bed,4.97 (195)
4,2022,Asheville,Home in Waynesville,122,1 king bed,New


# Datase 7

In [112]:
# datasets 7
# find the state abbre to match the state 
states = {
            'AK': 'Alaska',
            'AL': 'Alabama',
            'AR': 'Arkansas',
            'AS': 'American Samoa',
            'AZ': 'Arizona',
            'CA': 'California',
            'CO': 'Colorado',
            'CT': 'Connecticut',
            'DC': 'District of Columbia',
            'DE': 'Delaware',
            'FL': 'Florida',
            'GA': 'Georgia',
            'GU': 'Guam',
            'HI': 'Hawaii',
            'IA': 'Iowa',
            'ID': 'Idaho',
            'IL': 'Illinois',
            'IN': 'Indiana',
            'KS': 'Kansas',
            'KY': 'Kentucky',
            'LA': 'Louisiana',
            'MA': 'Massachusetts',
            'MD': 'Maryland',
            'ME': 'Maine',
            'MI': 'Michigan',
            'MN': 'Minnesota',
            'MO': 'Missouri',
            'MP': 'Northern Mariana Islands',
            'MS': 'Mississippi',
            'MT': 'Montana',
            'NA': 'National',
            'NC': 'North Carolina',
            'ND': 'North Dakota',
            'NE': 'Nebraska',
            'NH': 'New Hampshire',
            'NJ': 'New Jersey',
            'NM': 'New Mexico',
            'NV': 'Nevada',
            'NY': 'New York',
            'OH': 'Ohio',
            'OK': 'Oklahoma',
            'OR': 'Oregon',
            'PA': 'Pennsylvania',
            'PR': 'Puerto Rico',
            'RI': 'Rhode Island',
            'SC': 'South Carolina',
            'SD': 'South Dakota',
            'TN': 'Tennessee',
            'TX': 'Texas',
            'UT': 'Utah',
            'VA': 'Virginia',
            'VI': 'Virgin Islands',
            'VT': 'Vermont',
            'WA': 'Washington',
            'WI': 'Wisconsin',
            'WV': 'West Virginia',
            'WY': 'Wyoming'
    } 
# this data is from extract jupiter (one of the results)
abbre = ['CA', 'CO', 'FL', 'IL', 'LA', 'MA', 'MN', 'NV', 'NJ', 'NY', 'NC', 'OH', 'OR', 'RI', 'TN', 'TX', 'WA', 'DC']
results = []
for key in states: 
    if key in abbre:
        results.append(states[key])
print(results)


['California', 'Colorado', 'District of Columbia', 'Florida', 'Illinois', 'Louisiana', 'Massachusetts', 'Minnesota', 'North Carolina', 'New Jersey', 'Nevada', 'New York', 'Ohio', 'Oregon', 'Rhode Island', 'Tennessee', 'Texas', 'Washington']


In [113]:
crime.head()

,Year,State_Abbrevation,Popultion,Robbery,Homicide,Burglary,Motor Vehicle Theft
0,2020,CA,39368078,44728,2203,145529,168323
1,2020,CO,5807719,3964,294,23246,30452
2,2020,FL,21733312,13521,1290,52293,38271
3,2020,IL,12587530,12261,1151,31020,21332
4,2020,LA,4645318,3747,734,23698,12400


In [128]:
crime.head()

,Year,state_abbrevation,popultion,robbery,homicide,burglary,motor_vehicle_theft,state,robbery_rate,homicide_rate,burglary_rate,motor_vehicle_theft_rate
0,2020,CA,39368078,44728,2203,145529,168323,California,0.0011,0.0001,0.0037,0.0043
1,2020,CO,5807719,3964,294,23246,30452,Colorado,0.0007,0.0001,0.0040,0.0052
2,2020,FL,21733312,13521,1290,52293,38271,District of Columbia,0.0006,0.0001,0.0024,0.0018
3,2020,IL,12587530,12261,1151,31020,21332,Florida,0.0010,0.0001,0.0025,0.0017
4,2020,LA,4645318,3747,734,23698,12400,Illinois,0.0008,0.0002,0.0051,0.0027


In [161]:
# add the ID for crime datasets
#crime["id"] = crime.indsx + 1
# crime.set_index("id")
crime = crime.rename(columns = {"State_Abbrevation":"state_abbrevation",
                                   "Popultion":"popultion",
                               "Robbery" : "robbery",
                               "Homicide":"homicide",
                               "Burglary": "burglary",
                               "Motor Vehicle Theft": "motor_vehicle_theft"})
crime["state"] = results
crime["robbery_rate"] = round(crime["robbery"]/crime["popultion"],4)
crime["homicide_rate"] = round(crime["homicide"]/crime["popultion"],4)
crime["burglary_rate"] = round(crime["burglary"]/crime["popultion"],4)
crime["motor_vehicle_theft_rate"] = round(crime["motor_vehicle_theft"]/crime["popultion"],4)
crime.head()


# select the columne we need 
crime_new = crime[["state_abbrevation","state","popultion","robbery_rate","homicide_rate","burglary_rate","motor_vehicle_theft_rate"]]
crime_new.head(20)
crime_new

crime_data = {'state_abbrevation': 'HI', 'state': 'Hawaii', 'popultion': 3455100, 'robbery_rate': 0,'homicide_rate': 0,'burglary_rate':0,'motor_vehicle_theft_rate': 0}
crime_new = crime_new.append(crime_data, ignore_index = True)
crime_new.tail()
#crime_new.to_csv("crime.csv", index = True, header = True)

/var/folders/cl/3078jkyx5kq_gw3nfjk3j9tw0000gn/T/ipykernel_52435/1128471581.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crime_new = crime_new.append(crime_data, ignore_index = True)


,state_abbrevation,state,popultion,robbery_rate,homicide_rate,burglary_rate,motor_vehicle_theft_rate
14,TN,Rhode Island,6886834,0.0008,0.0001,0.0038,0.0031
15,TX,Tennessee,29360759,0.0009,0.0001,0.0037,0.0029
16,WA,Texas,7693612,0.0007,0.0000,0.0052,0.0036
17,DC,Washington,712816,0.0033,0.0003,0.0028,0.0048
18,HI,Hawaii,3455100,0.0000,0.0000,0.0000,0.0000


# Dataset 8

In [152]:
# add the ID
OSMR["sample_id"] = OSMR.index + 1
OSMR.set_index("sample_id")
OSMR["duration_min"] = round(OSMR["Duration"]/60,3)
OSMR["distance_m"] = OSMR["Distance"]
OSMR = OSMR.rename(columns = {"City":"city"})
OSMR_new = OSMR[["sample_id","city","duration_min","distance_m"]]
OSMR_new
#OSMR_new.to_csv("OSMR.csv", index = True, header = True)

,sample_id,city,duration_min,distance_m
0,1,New York City,23.525,18470.4
1,2,New Orleans,10.883,10099.6
2,3,New York City,22.503,11778.1
3,4,Rhode Island,626.168,857422.7
4,5,New York City,13.957,6822.5
...,...,...,...,...
170,171,Los Angeles,23.712,23347.9
171,172,Clark County,8.413,5797.3
172,173,Rhode Island,650.840,885725.8
173,174,Washington D.C.,15.968,10901.7
